In [1]:
# For grid creation
import os
import sys
import numpy as np
# run installed version of flopy or add local path
try:
    import flopy
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference
    from flopy.utils import Raster
    from flopy.utils.gridintersect import GridIntersect
except:
    import flopy
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference
    from flopy.utils import Raster
from flopy.utils.geometry import Polygon, LineString, Point
from flopy.utils.gridgen import Gridgen
from flopy.utils import OptionBlock
import flopy.utils.binaryfile as bf
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
import pyproj # for converting proj4string
import pandas as pd
import shapely
import geopandas as gpd
import calendar
import time
import rasterio

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

flopy is installed in C:\Users\mbkni\anaconda3\envs\geosp\lib\site-packages\flopy
3.8.5 (default, Aug  5 2020, 09:44:06) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.19.1
matplotlib version: 3.3.1
flopy version: 3.3.1


In [2]:
# Transient model
END_YEAR = 2020
STRT_YEAR = 2018

nper = (END_YEAR - STRT_YEAR)*365 
monlen = [] # length of stress period 
for year in range(STRT_YEAR, END_YEAR):
    for month in range(1,13):
        monlen.append(calendar.monthrange(year, month)[1])
monlen = np.array(monlen)
# nper = int(np.sum(monlen)) # Number of stress periods


perlen = np.ones(nper)
# Steady or transient periods
steady = np.zeros(nper)
steady[0] = 1 # first period is steady state, rest are transient
steady = steady.astype('bool').tolist()
nstp = np.ones(nper)*10

In [3]:
# Model parameters
nrow=80
ncol=230
delr=200
delc=200
rotation=44.7

# The number of layers should be 1 for the Mehrten formation, 1 for the laguna plus the number of TPROGS layers,
# where the Laguna formation will be clipped by the TPROGS layers
num_tprogs = 4
nlay = 2 + num_tprogs
tprog_thick = 25

# Model coordinate system

# proj4_str='EPSG:26910' # NAD83 UTM Zone 10N
proj4_str='+proj=utm +zone=10 +ellps=WGS84 +datum=WGS84 +units=m +no_defs '

In [19]:
# Import relevant files
m_domain = gpd.read_file(os.path.dirname(os.getcwd())+'\\GWModelDomain_UTM10N\\GWModelDomain_Rec_UTM10N.shp') #model domain
wells    = gpd.read_file('ProjectedWell_Elevations_m_geometry.shp') # well elevations in UTM Zone 10
wells

tsd_all = pd.read_csv('tsd_2018_2019.csv') 

xul, yul = list(m_domain.geometry.values[0].exterior.coords)[3]
list(m_domain.geometry.values[0].exterior.coords)
# m_domain.geometry.values[0].exterior

[(661606.9378817221, 4272495.737420673),
 (672690.3174606431, 4260899.797577698),
 (638997.1522573363, 4229460.230514871),
 (627913.7726784144, 4241056.170357846),
 (661606.9378817221, 4272495.737420673)]

In [20]:
m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'MODFLOW-NWT', 
                          version = 'mfnwt', model_ws='data')
# m = flopy.modflow.Modflow(modelname = 'MF', exe_name = 'mf2005', 
#                           version = 'mf2005', model_ws='data')
#lenuni = 1 is in ft, lenuni = 2 is in meters
# itmuni is time unit 5 = years, 4=days, 3 =hours, 2=minutes, 1=seconds
dis = flopy.modflow.ModflowDis(nrow=nrow, ncol=ncol, 
                               nlay=nlay, delr=delr, delc=delc,
                               model=m, lenuni = 2, itmuni = 4,
                               xul = xul, yul = yul,rotation=rotation, proj4_str=proj4_str,
                              nper = nper, perlen=perlen, nstp=nstp, steady = steady,
                              start_datetime = '1/1/2017')

In [21]:
# m.modelgrid.xyzextent
# m.modelgrid.extent
# m.modelgrid.shape
# m.modelgrid.nnodes
# m.modelgrid.grid_lines
# np.shape(m.modelgrid.xcellcenters)
# m.modelgrid.xcellcenters[0]
# m.modelgrid.get_local_coords(639168.0879233824, 4229683.378775878)
# m.modelgrid.get_local_coords(627913.7726784144, 4241056.170357846)
# m.get_nrow_ncol_nlay_nper

# mg = m.modelgrid
# ll = mg.get_coords(0, 0) #lower left
# lr = mg.get_coords(0, nrow*delr) #lower right
# ur = mg.get_coords(ncol*delc, nrow*delr) #upper right
# ul = mg.get_coords(ncol*delc, 0) #upper left
# print(ll, lr, ur, ul)

# mg.plot()
# mg.get_coords(0,0)

# os.getcwd()

# # Get vertexes of model domain
# # ll = mg.get_coords(0, 0) #lower left
# # lr = mg.get_coords(nrow*delr, 0) #lower right
# # ur = mg.get_coords(nrow*delr, ncol*delc) #upper right
# # ul = mg.get_coords(0, ncol*delc) #upper left


# # Shapefile of model bounds
# from shapely.geometry import Polygon
# vertices = np.stack(np.asarray((ll,lr, ur, ul)))
# vertices
# geoms = Polygon(vertices)

In [22]:
# create grid intersect object to apply spatial data to modflow grid
mg_intersect = GridIntersect(m.modelgrid, method = 'structured')#'WellsProjected.shp

# Find the intersect of spatial points with the grid, returns the cell IDs of the modflow cells
vertices = mg_intersect.intersect_point(wells.geometry.values)#gcoords_new.geometry.values)
vertices

rec.array([((39, 141), <shapely.geometry.point.Point object at 0x000001A7AB0A7E80>),
           ((33, 140), <shapely.geometry.point.Point object at 0x000001A7AB0A76D0>),
           ((31, 167), <shapely.geometry.point.Point object at 0x000001A7AB0A75B0>),
           ((35, 168), <shapely.geometry.point.Point object at 0x000001A7AFD9A1C0>),
           ((34, 151), <shapely.geometry.point.Point object at 0x000001A7AFD9A1F0>),
           ((36, 152), <shapely.geometry.point.Point object at 0x000001A7AFD9A040>),
           ((50, 39), <shapely.geometry.point.Point object at 0x000001A7AFD9A3D0>),
           ((48, 51), <shapely.geometry.point.Point object at 0x000001A7AFD9A340>),
           ((48, 47), <shapely.geometry.point.Point object at 0x000001A7AFD9A370>),
           ((37, 95), <shapely.geometry.point.Point object at 0x000001A7AFD9A4C0>),
           ((52, 51), <shapely.geometry.point.Point object at 0x000001A7AFD9A520>),
           ((32, 171), <shapely.geometry.point.Point object at 0x00000

In [23]:
# len(vertices)
# 80 rows, 230 columns
# vertices[index-0 to 23][first tuple=0][row=0/column=1]
# a = vertices[0][0][0]#[0]#[0][0]#[0]
# a
# b = len(vertices)
# b
# c = wells.Well_name.values[1]
# c

# data = []
# data=[a,b,c]
# data.append(data)
# data
# #np.shape(data)

In [24]:
tsd_all.columns
# i=1
#tsd_all[wells.Well_name[0]]

Index(['Datetime', 'MW_5', 'UCD_26', 'MW_19', 'Rooney_1', 'MW_11', 'MW_22',
       'MW_9', 'MW_405', 'MW_218', 'MW_DR1', 'MW2', 'MW3', 'MW7', 'MW13',
       'MW14', 'MW17', 'MW20', 'MW23', 'OnetoAg', 'MWCP1', 'MWT_1', 'MWT_2',
       'MWR_1', 'MWR_2', 'T_5', 'T_6'],
      dtype='object')

In [25]:
data = []
for i in range(len(vertices)):
    data_temp=[1, vertices[i][0][0], vertices[i][0][1], tsd_all[wells.Well_name[i]], wells.Well_name[i]]
    data.append(data_temp)
data

[[1,
  39,
  141,
  0          NaN
  1          NaN
  2          NaN
  3          NaN
  4          NaN
  5          NaN
  6          NaN
  7          NaN
  8          NaN
  9          NaN
  10         NaN
  11         NaN
  12         NaN
  13         NaN
  14         NaN
  15   -2.476035
  16   -2.714825
  17   -3.075355
  18   -4.225005
  19   -4.722667
  20   -4.633728
  21   -4.194095
  22   -3.843021
  23   -3.247583
  Name: MWT_1, dtype: float64,
  'MWT_1'],
 [1,
  33,
  140,
  0           NaN
  1           NaN
  2           NaN
  3           NaN
  4           NaN
  5           NaN
  6           NaN
  7           NaN
  8           NaN
  9           NaN
  10          NaN
  11          NaN
  12          NaN
  13          NaN
  14          NaN
  15    16.534706
  16    16.343622
  17    15.845969
  18    14.639412
  19    13.774289
  20    13.111947
  21    12.537265
  22    11.995586
  23    12.675420
  Name: MWT_2, dtype: float64,
  'MWT_2'],
 [1,
  31,
  167,
  0          NaN
  1

In [59]:
tsd_sp = list(np.arange(1,25))
tsd_ob = list(tsd_all.MW_5.values)
tsd_sp
tsd_ob
zipped = zip(tsd_sp, tsd_ob)
tsd = list(zipped)

In [ ]:
data = []
for i in range(len(vertices)):
    data_temp=[1, vertices[i][0][0], vertices[i][0][1], tsd_all[wells.Well_name[i]], wells.Well_name[i]]
    data.append(data_temp)
data

In [105]:
wells.Well_name[i]

'MW13'

In [ ]:
vertices[i][0][0]

In [106]:
# create a new hob object
obs_data = []

for i in range(len(vertices)):
# observation location 1
    tsd_sp = list(np.arange(1,25))
    tsd_ob = list(tsd_all.UCD_26.values)
    tsd = [list(a) for a in zip(tsd_sp, tsd_ob)]
    obs_data.append(flopy.modflow.HeadObservation(m, layer=0, row=vertices[i][0][0], column=vertices[i][0][1],
                                                  time_series_data=tsd,
                                                  obsname=wells.Well_name[i]))
# # observation location 2
# tsd_sp = list(np.arange(1,25))
# tsd_ob = list(tsd_all.MWT_1.values)
# tsd = [list(a) for a in zip(tsd_sp, tsd_ob)]
# obs_data.append(flopy.modflow.HeadObservation(m, layer=0, row=3, column=3,
#                                               time_series_data=tsd,
#                                               #names=names, 
#                                               obsname='MWT_1'))
hob = flopy.modflow.ModflowHob(m, iuhobsv=51, hobdry=-9999., obs_data=obs_data)

In [109]:
obs_data

In [114]:
obs_data[2].time_series_data

rec.array([( 1.,  1, 1.1102230e-16,         nan, b'MWR_2.1'),
           ( 2.,  1, 1.0000000e+00,  3.7171009 , b'MWR_2.2'),
           ( 3.,  2, 1.0000000e+00,  4.0312138 , b'MWR_2.3'),
           ( 4.,  3, 1.0000000e+00,  4.314156  , b'MWR_2.4'),
           ( 5.,  5, 1.7763568e-15,  3.9448776 , b'MWR_2.5'),
           ( 6.,  6, 5.3290705e-15,  3.4830081 , b'MWR_2.6'),
           ( 7.,  7, 8.8817842e-15,         nan, b'MWR_2.7'),
           ( 8.,  8, 1.2434498e-14, -0.3032927 , b'MWR_2.8'),
           ( 9.,  9, 1.5987212e-14,         nan, b'MWR_2.9'),
           (10., 10, 1.9539925e-14,         nan, b'MWR_2.10'),
           (11., 11, 2.3092639e-14,         nan, b'MWR_2.11'),
           (12., 12, 2.6645353e-14,         nan, b'MWR_2.12'),
           (13., 13, 3.0198066e-14,         nan, b'MWR_2.13'),
           (14., 14, 3.3750780e-14,         nan, b'MWR_2.14'),
           (15., 15, 3.7303494e-14,         nan, b'MWR_2.15'),
           (16., 16, 4.0856207e-14,         nan, b'MWR_2.16'),
 

In [ ]:
# in the shapefile have x, y, pumping (at some point figure out a way to choose the layer based on the well depth
# ask someone to make a spreadsheet with well depth included in the document
# import shapefile
# get data in the right format for the hob file: layer, row, column, tsd (do I need to join based on well name and then drop the name?)
# figure out how to get the right stress period number - figured it out, it is whatever i define for stress periods. since this may
# change it might be useful to find a way to automate this eventually

# totim = total simulation time

# IREFSP — is the stress period to which the observation time is referenced. The reference point is the 
# beginning of the specified stress period. If the value of IREFSP read in item 3 is negative, there are 
# observations at |IREFSP| times -- item 5 is read and |IREFSP| repetitions of item 6 are read. Also, if 
# IREFSP is negative, values of OBSNAM, TOFFSET, HOBS, STATISTIC, STAT-FLAG and PLOT-SYMBOL read in item 3 
# are ignored and values read in item 6 are used.